<a href="https://colab.research.google.com/github/Adhvik09-oss/Adhvik09-oss/blob/main/SnapCareipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing out neccessary modules
import torch as nn
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision
import torch
import torchvision.models as models
import torch.optim as optim


In [2]:
#writing our device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

### for more information on the different transfer learning architectures look at:
https://docs.pytorch.org/vision/0.9/models.html

In [3]:
resnet = models.resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 174MB/s]


In [4]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
import torch.nn as nn
# we have to change the final layer because our dataset(SD-198) has 198 classes, unlike the standard 100 that resnet-50 comes with
resnet.fc = nn.Linear(in_features=2048, out_features=198)

In [6]:
#freeze all the layers so we won't have to train it fully
for param in resnet.parameters():
  param.requires_grad = False

#unfreeze the fully connected layer
for param in resnet.fc.parameters():
  param.requires_grad = True

In [7]:
#selecting our loss function and optimizers
loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params = resnet.parameters(),
                            lr = 0.01,
                            momentum = 0.09)

In [8]:
#importing our dataset and preparing to create a dataloader
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import zipfile
import os


skin_data_path = '/content/drive/MyDrive/data.zip'
extract_path = '/content/SD-198'  # Local Colab path to unzip

with zipfile.ZipFile(skin_data_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [19]:
from torchvision import transforms
from torch.utils.data import DataLoader,random_split # import random split for later use
from torchvision.datasets import ImageFolder

train_data = transforms.Compose([
    transforms.Resize((224,224)), #resizes img to 224x224 which is what resnet uses
    transforms.ToTensor(), #transforms from PIL to Pytorch Tensor
    transforms.Normalize([0.485,0.456,0.406],
                          [0.229, 0.224, 0.225])

])

dataset = ImageFolder(root = extract_path, transform = train_data)
dataloader = DataLoader(dataset, batch_size = 32, shuffle = True)

In [49]:
#create training/testing split

train_size = int(0.8*len(dataloader))
test_size = len(dataloader) - train_size

training_data,testing_data = random_split(dataloader,[train_size,test_size])

#create the training and testing data_loaders

train_loader = DataLoader(training_data,batch_size = 32, shuffle = True)
test_loader = DataLoader(testing_data,batch_size = 32,shuffle = False)

In [52]:
dataiter = iter(train_loader)
images, labels = next(dataiter)


TypeError: 'DataLoader' object is not subscriptable

In [50]:
import matplotlib.pyplot as plt
import numpy as np

# Get one batch from the DataLoader
dataiter = iter(train_loader)  # <- make sure you're using `train_loader`
images, labels = next(dataiter)

# Function to unnormalize and show an image
def imshow(img):
    img = img.numpy().transpose((1, 2, 0))  # CHW to HWC
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean  # Unnormalize
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Show the first image in the batch
imshow(images[0])
print(f"Label: {labels[0]}")


TypeError: 'DataLoader' object is not subscriptable